# Prep des macro-domaines DUFLT

Version février 2026 intégrant les 40 domaines de Denis

In [126]:
import pandas as pd
import os

import datetime

# date as "yyyymmdd"
suffix = datetime.datetime.now().strftime("%y%m%d")
suffix

'260211'

In [112]:
filename = "Trevoux1743_paragraphs_260210.tsv"

df = pd.read_csv(os.path.join('..', 'data', '1743_LeRobert', filename), sep='\t', encoding='utf-8')
df.head()

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain
0,DUFLT_1743,1,1,A,1,mainEntry,1,NaN,A est la première Lettre de l'Alphabet Françoi...,0
1,DUFLT_1743,1,1,A,1,mainEntry,2,NaN,C'est inutilement que la plupart des Grammairi...,0
2,DUFLT_1743,1,1,A,1,mainEntry,3,NaN,"A se prononce du gozier, ce qui ne rend pas ce...",0
3,DUFLT_1743,1,1,A,1,mainEntry,4,NaN,Le son de l'a est ordinairement un son clair. ...,0
4,DUFLT_1743,1,1,A,1,mainEntry,5,NaN,Le son de l'a est un de ceux que les muets for...,0


In [113]:
df.shape

(112053, 10)

In [117]:
df_domaines_denis = pd.read_excel('Conversion_DUFLT_protodesignants_MacroDomaines_Denis.xlsx')
df_domaines_denis.rename(columns={"subordinate_domain":"srcDomain", "Macro-domaines correspondants": "macroDomain"}, inplace=True)
df_domaines_denis = df_domaines_denis[["srcDomain", "macroDomain"]]

df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace('histoire', 'Histoire')
df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace(' Religion', 'Religion')
df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace('Métiers [ métal, minéral,  dérivés]', 'Métiers [métal, minéral, dérivés]')
df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace('Agriculture - Économie domestique', 'Agriculture - Économie rustique')

df_domaines_denis.head()

/Users/lmoncla/git/github.com/lmoncla/trevoux/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,srcDomain,macroDomain
0,Terme d'Agriculture,Agriculture - Économie rustique
1,Terme d'Agriculture & de Coutumes,Agriculture - Économie rustique
2,Terme d'Agriculture & de Jardinage,Agriculture - Économie rustique
3,Terme d'Agriculture & de labourage,Agriculture - Économie rustique
4,Terme d'Agriculture & de Vigneron,Agriculture - Économie rustique


In [118]:
print(df_domaines_denis['macroDomain'].unique())
print(len(df_domaines_denis['macroDomain'].unique()))

['Agriculture - Économie rustique' 'Anatomie' 'Antiquité' 'Architecture'
 'Arts mécaniques - Manufactures' 'Beaux-arts' 'Belles-lettres' 'Blason'
 'Chasse-pêche' 'Chimie' 'Commerce' 'Droit Jurisprudence' 'Géographie'
 'Grammaire' 'Histoire' 'Histoire naturelle' 'Jeu' 'Maréchalerie - Manège'
 'Marine' 'Mathématiques' 'Médailles' 'Médecine' 'Mesure'
 'Métiers [métal, minéral, dérivés]' 'Métiers [alimentation]'
 'Métiers [autres]' 'Métiers [bois]' 'Métiers [cuir, peaux]'
 'Métiers [papier]' 'Métiers [tissu, habit]' 'Militaire' 'Monnaie'
 'Musique' 'Pharmacie' 'Philosophie' 'Religion'
 'Sciences physico-mathématiques' 'Superstition' 'Voyage']
39


In [119]:
# lowercase src-domain columns
df_domaines_denis['srcDomain'] = df_domaines_denis['srcDomain'].str.lower()
df['srcDomain'] = df['srcDomain'].str.lower()

In [120]:
#TODO: improve matching by stripping accents, etc...
import unicodedata

def remove_accents(text):
    if not isinstance(text, str):
        return text
    # On normalise le texte en forme NFD (Normal Form Decomposition)
    # Cela sépare le caractère de son accent
    texte_normalise = unicodedata.normalize('NFD', text)
    
    # On filtre pour ne garder que les caractères qui ne sont pas des "marques" (accents)
    # Les caractères de type 'Mn' correspondent aux "Mark, Nonspacing"
    return "".join(c for c in texte_normalise if unicodedata.category(c) != 'Mn')

def remove_terme_de(text):
    if not isinstance(text, str):
        return text
    text = text.replace('terme de la ', '')
    text = text.replace("terme de l'", '')
    text = text.replace('terme des ', '')
    text = text.replace('terme de ', '')
    text = text.replace('terme du ', '')
    text = text.replace("terme d'", '')
    text = text.replace('termes de la ', '')
    text = text.replace("termes de l'", '')
    text = text.replace('termes des ', '')
    text = text.replace('termes de ', '')
    text = text.replace('termes du ', '')
    text = text.replace("termes d'", '')
    text = text.replace("termes", '')
    text = text.replace("terme", '')
    return text

df_domaines_denis['srcDomain'] = df_domaines_denis['srcDomain'].apply(remove_accents)
df['srcDomain'] = df['srcDomain'].apply(remove_accents)

df_domaines_denis['srcDomain'] = df_domaines_denis['srcDomain'].apply(remove_terme_de)
df['srcDomain'] = df['srcDomain'].apply(remove_terme_de)


In [121]:
# remove duplicates in df_domaines_denis
df_domaines_denis = df_domaines_denis.drop_duplicates(subset=['srcDomain'])

In [122]:
df = pd.merge(df, df_domaines_denis, on='srcDomain', how='left')
df.head()

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain,macroDomain
0,DUFLT_1743,1,1,A,1,mainEntry,1,NaN,A est la première Lettre de l'Alphabet Françoi...,0,NaN
1,DUFLT_1743,1,1,A,1,mainEntry,2,NaN,C'est inutilement que la plupart des Grammairi...,0,NaN
2,DUFLT_1743,1,1,A,1,mainEntry,3,NaN,"A se prononce du gozier, ce qui ne rend pas ce...",0,NaN
3,DUFLT_1743,1,1,A,1,mainEntry,4,NaN,Le son de l'a est ordinairement un son clair. ...,0,NaN
4,DUFLT_1743,1,1,A,1,mainEntry,5,NaN,Le son de l'a est un de ceux que les muets for...,0,NaN


In [123]:
df.shape

(112053, 11)

In [124]:
# macro_domain not NaN
df[df['srcDomain'].notna()]

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain,macroDomain
62,DUFLT_1743,1,15,ABADIR,1,mainEntry,1,mythologie,"ABADIR ou ABADDIR ; car Priscien, qui nous a c...",1,Antiquité
66,DUFLT_1743,1,16,ABAEUZ,1,mainEntry,1,coutume,ABAEUZ Terme de Coutume . Biens abaeuz. Bona v...,1,Droit Jurisprudence
67,DUFLT_1743,1,17,ABAJOUR,1,mainEntry,1,architecture,"ABAJOUR Terme d'Architecture , Spiraculum, esp...",1,Architecture
69,DUFLT_1743,1,17,ABAJOUR,2,relatedEntry,1,botanique,ABAJOUR Abajour . s. m. Terme de Botanique . S...,2,Histoire naturelle
71,DUFLT_1743,1,19,ABAISSE,1,mainEntry,1,patissier,ABAISSE terme de Pâtissier . C'est la pâte qui...,1,Métiers [alimentation]
...,...,...,...,...,...,...,...,...,...,...,...
112020,DUFLT_1743,6,5560,ZULUFDGILER,1,mainEntry,1,relation,ZULUFDGILER Terme de Relation . Enfant de trib...,1,Voyage
112023,DUFLT_1743,6,5563,ZUNDANASTAN,1,mainEntry,1,relation,ZUNDANASTAN Terme de Relation . C'est le livre...,1,Voyage
112025,DUFLT_1743,6,5565,ZUNITTER,1,mainEntry,1,philosophie hermetique,ZUNITTER ou ZITTER. s. m. Terme de Philosophie...,1,Philosophie
112027,DUFLT_1743,6,5567,ZURANDALE,1,mainEntry,1,fleuriste,ZURANDALE Terme de Fleuriste . Nom de tulipe. ...,1,Métiers [autres]


In [125]:
df[df['srcDomain'].notna() & ~df['macroDomain'].notna()]

,book,volume,numero,head,subEntryId,type,paragraphId,srcDomain,text,numUsgDomain,macroDomain
254,DUFLT_1743,1,108,ABIGEAT,1,mainEntry,1,droit romain,ABIGEAT Terme de Droit Romain . L'Abigeat est ...,1,NaN
283,DUFLT_1743,1,129,ABOKELLE,1,mainEntry,1,negotiant en egypte & de relation,ABOKELLE Terme de Négotiant en Égypte & de Rel...,1,NaN
288,DUFLT_1743,1,132,ABOLITION,1,mainEntry,1,chancellerie,ABOLITION Terme de Chancellerie . Abolitio cri...,1,NaN
399,DUFLT_1743,1,182,ABREUVER,3,relatedEntry,1,vernisseur,"ABREUVER Abreuver , verbe actif, terme de Vern...",1,NaN
451,DUFLT_1743,1,208,ABSIDE,1,mainEntry,1,architecture & de liturgie,ABSIDE Absis & absida. Terme d'Architecture & ...,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
110976,DUFLT_1743,6,4745,USER,9,relatedEntry,3,forets,"En termes de Forêts on appelle vente usée, cel...",1,NaN
111062,DUFLT_1743,6,4795,VÛE,16,relatedEntry,2,lettres de change,"En termes de Lettres de Change , on dit, qu'un...",1,NaN
111095,DUFLT_1743,6,4799,VUIDER,5,relatedEntry,1,artisans,"VUIDER Vuider , en termes d'Artisans , signifi...",1,NaN
111101,DUFLT_1743,6,4800,"VULCAIN, VOLCAN",2,relatedEntry,1,hermetique,"VULCAIN, VOLCAN Le Vulcain des Philosophes , e...",1,NaN


In [127]:
# Liste des src-domain qui n'ont pas de macro-domain correspondant
l = list(df[df['srcDomain'].notna() & ~df['macroDomain'].notna()]['srcDomain'].unique())

# strip whitespace from values in l
l = [x.strip() for x in l]

# sort l: fist value without &, then values with & and sorted alphabetically
l = sorted(l, key=lambda x: (('&' in x), x))

print(len(l))

# put this as a first column in a csv file for manual annotation
pd.DataFrame(l, columns=['srcDomain']).to_excel(os.path.join('..', 'data', '1743_LeRobert', f'src_domains_without_macro_{suffix}.xlsx'), index=False)

565


In [128]:
path = os.path.join('..', 'data', '1743_LeRobert')
df.to_csv(os.path.join(path, "Trevoux1743_paragraphs_macro_"+suffix+".tsv"), sep="\t", index=False)
df.to_excel(os.path.join(path, "Trevoux1743_paragraphs_macro_"+suffix+".xlsx"), index=False)

In [ ]:
df.shape